Here we are finally at the final project. As you know, we are going to extract
information on the course of the cryptomonnages and see if the feeling of tweets expressed in relation to these cryptomonnages correlates with
the evolution of their price.

Of course, we're not going to code everything from the beginning. We can reuse parts of the code that other people have made public to
be able to accelerate our development. These code parts are
usually called "bookstores" and identify which ones are useful
and how they work are often an important part of a project (especially for smaller projects).

In this notebook, we'll see how the libraries you'll have
needed for the project is working. Of course, we will only show here
that certain functions you will probably have to use.
This list is not exhaustive and you will still need to look for
information on bookstores and functions on the internet. By
elsewhere, if you prefer to use other libraries for the project,
you're obviously free to do so.

### Requests and APIs

A good part of the project consists in finding information through
of API. In this section, we see what APIs are and how they work.
use them with the *requests* library.

### What's an API?

An API is the acronym for Application Programming Interface. It is actually
a set of instructions that the designers of an application (whether local or online) made available to other programmers who would like to use their application through code-based interactions (i.e. without going through the GUI).

There are many APIs for a whole bunch of sites or applications. For the project, you will need to use the coinpaprika and twitter API. For
using the first API, we can use a very powerful library of
who just makes a call to the site and gets an answer. For the Twitter API, the fact that you have to identify yourself makes the process more
complex and, for this reason, we will use a library that manages
most of the authentication details for us.

Here's a video that explains what a web API is (there are also
Local APIs, but we won't see them in this course). *The company that produced this video sells integration solutions, so the last part of the video is largely an ad but the video is
instructive if you've never used an API before.

http://www.youtube.com/watch?feature=player_embedded&v=s7wmiS2mSXY

#### The Game of Thrones API

To use an API, you usually need to know what you want to
recover. During this session, and because it's in season, we're going to
use an API that allows us to access information about [Game of Thrones characters and their h](https://anapioficeandfire.com/)ouses. In this saga, characters from several families (called
houses) compete to become the Queen or King of the 7 Kingdoms, and
govern Westeros. Dragons are also part of the picture. *If you have a worse way of describing it, let me know *.

This kind of API may seem a bit pointless as it is. However, it is
possible to couple this API with others and to program a little to get [this kind of result](https://got.show/) - a predictive model giving the likelihood that each character will die in the last season (a project from the Munich Technical University).

The first step, after identifying the API you want to use is to read the documentation. Fortunately, in the case of the API for
GoT (we will use this abbreviation for Games of Thrones in the
rest of this notebook), it's pretty simple. The documentation can be
can be found [on this page](https://anapioficeandfire.com/Documentation).

On this page you can find some important information:

1. We will not be able to receive information from more than 50 characters or houses per API call.
2. There are two interesting "features" for us
    - The one that lists the characters ([https://www.anapioficeandfire.com/api/characters](https://www.anapioficeandfire.com/api/characters) + options)
    - The one that lists the houses ([https://www.anapioficeandfire.com/api/houses](https://www.anapioficeandfire.com/api/houses) + options)

The Requests bookstore

To use simple APIs like this one (and the one from Coinpaprika), you can use the *requests* library. This library manages the request forwading to the server, its reception and additionnally deals with formatting. Let's try to load the first outcome page consisting in all of the GoT characters. 

Let's start by importing the requests library.



In [ ]:
import requests

We will now write the string containing the address that the documentation gives as the one to get the character list. We store it in a variable and then pass it through the
string to the get() function from the requests library.

In [ ]:
request_string = "https://www.anapioficeandfire.com/api/characters"
response = requests.get(request_string)

By default, nothing is returned, but you can display the contents of the response.

In [ ]:
response.json()[0:3]

Here, we must use the .json() method because response type objects are complex and contain several elements.

*[{'url': 'https://www.anapioficeandfire.com/api/characters/1', 'name': '', 'gender': 'Female', 'culture': 'Braavosi', 'born': '', 'died': '', 'titles': [''], 'aliases': ['The Daughter of the Dusk'], 'father': '', 'mother': '', 'spouse': '', 'allegiances': [], 'books': ['https://www.anapioficeandfire.com/api/books/5'], 'povBooks': [], 'tvSeries': [''], 'playedBy': ['']}, {'url': 'https://www.anapioficeandfire.com/api/characters/2', 'name': 'Walder', 'gender': 'Male', 'culture': '', 'born': '', 'died': '', 'titles': [''], 'aliases': ['Hodor'], 'father': '', 'mother': '', 'spouse': '', 'allegiances': ['https://www.anapioficeandfire.com/api/houses/362'], 'books': ['https://www.anapioficeandfire.com/api/books/1', 'https://www.anapioficeandfire.com/api/books/2', 'https://www.anapioficeandfire.com/api/books/3', 'https://www.anapioficeandfire.com/api/books/5', 'https://www.anapioficeandfire.com/api/books/8'], 'povBooks': [], 'tvSeries': ['Season 1', 'Season 2', 'Season 3', 'Season 4', 'Season 6'], 'playedBy': ['Kristian Nairn']}, {'url': 'https://www.anapioficeandfire.com/api/characters/3', 'name': '', 'gender': 'Male', 'culture': '', 'born': '', 'died': '', 'titles': [''], 'aliases': ['Lamprey'], 'father': '', 'mother': '', 'spouse': '', 'allegiances': ['https://www.anapioficeandfire.com/api/houses/15'], 'books': ['https://www.anapioficeandfire.com/api/books/3'], 'povBooks': [], 'tvSeries': [''], 'playedBy': ['']}]*

As we can see, the function returned a list of 10 characters (which do not have names, this is due to the fact that, the characters being
ranked by alphabetical order, the characters whose names are
empty appear first. We'll exclude these characters a little
later).

One problem with this result is that it only returns 10 characters. We want them all. So we'll have to create a loop that
continues to request additional pages until the answer is the
last. Since the logic is the same for the characters and the
houses, we're going to put that part of the code into a function.

In [ ]:
def get_GoT_info(root, start, end): # The argument is the root of the address which indicates whether we deal with the characters or the houses. 
final_list = [] # Create an empty list 
page_size = 50 # We take the maximum number of objects per call to avoid too many API calls. 
options = {"page": "1", "pageSize": str(page_size)} # The options for the first page, we need them to know how many pages we should extract. 
first_response = requests.get(root, params= options) # This line retrieves the first response. In this response, one of the fields in the "headers" contains the number of the last info page. 
# If you don't understand the next line, it's not too bad. They are specific to the GoT API and are not really relevant to the course. 
# If you want to understand why this is necessary, feel free to ask the assistant or send us a mail 
last_page = int(first_response.headers["Link"] [-start:-end]) 
for ii in range(1,last_page+1): 
	options = {"page": str(ii), "pageSize": str(page_size)} 
	<response = requests.get(root, params=options) 
	final_list = final_list + response.json() 
	return final_list

The function is a bit weird but it first checks the number of content pages available on the site (you never know they add, they might add characters or houses during the last episodes). Next, for each page, we ask the site to send us the list of the 50
characters or houses on this page. The characters or houses are
stored in a list (named *response*) and finally this list is added to the resulting list of all previous iterations.

We can now create two lists. One containing all the dictionaries representing the characters and the other one containing the dictionaries
containing the houses.

In [ ]:
characters = get_GoT_info("https://www.anapioficeandfire.com/api/characters", 27, 25) 
# Numbers 27, 26 and 25 were determined by inspecting the chains manually
houses = get_GoT_info("https://www.anapioficeandfire.com/api/houses", 26, 25) 
# A good exercise (but one that requires a lot of additional study) would be to automatically determine the number to be extracted. This requires the use of regular expressions (which we will not see in this course).

So much for the Game of Thrones API. Now let's see a short example of how to use the Twitter API.

The Twitter API

Some APIs are freely available, such as the GoT API. You can easily query them with *requests*. For Twitter, it's a bit more complicated. You need credentials.

You can create usernames quite simply: start by creating an
Twitter account (if no member of your group has one). Make an appointment
then on the [Twitter for developers] site ([https://developer.twitter.com/](https://developer.twitter.com/)). Then create a new application. This should give you access to 4 codes identifying your account and application. These codes are
Meaningless sequences of letters and numbers.

To use the Twitter API, we will use the [Python-Twitter] library ([https://python-twitter.readthedocs.io/en/latest/](https://python-twitter.readthedocs.io/en/latest/)). Using this library, you just have to give the credentials you received from Twitter and make requests.

In [ ]:
api_key = "5oxVSi0q8FWGKSZ68D2iwnFLZ"
api_secret_key = "51esJc1fCtAWpgBPd6TVr4Lun5QbtjI6afOJ00uPBD1x74L8eL"
access_token = "4143588544-7mu5NDQ7x1xa1hoIsZn2sT93GI01zbBDVue634c"
access_secret_token = "qQqrfGwuEduRZ5uAqekq72IoqkdDbdfsZVjB4NiV17wSJ"

We can now create an *API* object from the library. This object (we haven't seen the objects during this class, this would probably be the next topic you could study if you want to go further in the concepts) allows us to make queries to Twitter which can send us a copy of the tweets according to certain criteria.

A detailed understanding of the library requires reading in detail
documentation but, for simple queries, you can use
the following workflow: create an API object $\rightarrow$ define a string representing the search $\rightarrow$ send it to the *getSearch()* method of the API object $\rightarrow$ process the resulting list.

In [ ]:
import twitter
api = twitter.Api(consumer_key=api_key, consumer_secret=api_secret_key, access_token_key=access_token,  access_token_secret=access_secret_token)

The above code creates the API object. We can now create the request. In that case, we'll just search all the tweets regarding the houses of Stark, Lannister and Targaryen.

In [ ]:
main_houses = ["Stark", "Lannister", "Targaryen"]

Now we need to format a search string. To do this, you can use the search page in the following way: use the advanced search tool,
specifying the parameters you want. Then copy the parameter part of the URL.

In [ ]:
query_chain = ["House"+name for name in main_houses]

In [ ]:
number_tweets = 99 # This is the maximum number of tweets that can be requested with the free version of API
tweets_on_houses = {} # We create an empty dictionary for ii in query_chain: tweets = api.GetSearch(raw_query="l=en&q=%23"+ii+"&count="+str(number_tweets)) tweets_on_houses[ii] = tweet

In [ ]:
tweets_on_houses["HouseLannister"][0].id

In [ ]:
tweets_on_houses["HouseLannister"][0:5] # Frankly, Lannister's deserve the Iron throne.

*[Status(ID=1121038719195275265, ScreenName=YSoSeriousBoo, Created=Wed Apr 24 13:10:33 +0000 2019, Text='@Oreo @oreo #ForTheThrone #GOT \n #GameOfCookies \n\n#WhiteWalker 7 Oreos \n\n#HouseLannister 6 Oreos\n\n#HouseDargaryen… https://t.co/zgva8sb28N'), Status(ID=1121012886875385859, ScreenName=DohertyCiaran, Created=Wed Apr 24 11:27:54 +0000 2019, Text='Second of Six #GlassOfThrones stained glass installations unveiled at #Belfast Waterfront Hall. Depicting… https://t.co/eVttTzPSQD'), Status(ID=1120944289301843968, ScreenName=TourismIreland, Created=Wed Apr 24 06:55:19 +0000 2019, Text='RT @VirginMediaNews: PICTURED: Another #GameOfThrones-themed stained glass window has been unveiled in #Belfast as part of @TourismIreland’…'), Status(ID=1120944181906690049, ScreenName=SineadGrace1, Created=Wed Apr 24 06:54:53 +0000 2019, Text='RT @VirginMediaNews: PICTURED: Another #GameOfThrones-themed stained glass window has been unveiled in #Belfast as part of @TourismIreland’…'), Status(ID=1120908255193071617, ScreenName=opeadigs, Created=Wed Apr 24 04:32:08 +0000 2019, Text='RT @PiggyBankNG: What house do you belong to, based on the feature you use the most?\n\n#HouseTargaryen #HouseGreyJoy #HouseStark #HouseLanni…')]*

Nice!

## Interact with the database

In the first part of the project, you created a database.
This will be used to store all the information you will need for your analyses.

In the case of our GoT analysis, we need to create the database
specifically to receive the information we extracted. However, the creation procedure of a database and inserting values into it is more or less the same from a Python point of view.

Everything happens with *sqlite3* library. This library allows you to interact with a sqlite database by forwarding it strings containing the SQL statements we want to execute.

Let's start by importing the library...

In [ ]:
import sqlite3

Create the tables

The simplest way is to create a database (which can start as an empty file with a name ending in *.db* or *.sqlite*) in the folder where your Python code is located (whether it's a code in pure" Python - a ".py" - or a Jupyter notebook - a ".ipynb").

Once the file is created, you can use the sqlite3 library to handle it . The library documentation is available [here](https://docs.python.org/3/library/sqlite3.html).

The first step is to link to the data. This is created by passing the database name as a string to the *connect()* method of the library. In our case, we created a database called "GoT.db" in the folder containing this notebook. So we create a connection.

In [ ]:
conn = sqlite3.connect("GoT.db") # If the file doesn't exist, it is automatically created

Now that we have a connection, we can use the *execute()*, passing as argument a string of characters containing the instruction you wish to execute. In our case, we want to create two tables: one table containing the houses and a table containing the characters.

Some characters can have several allegiances, it is necessary therefore to create a join table (a house can have several members and a character may have pledged allegiance to more than one member. houses).

In [ ]:
create_houses = '''CREATE TABLE IF NOT EXISTS houses( url TEXT PRIMARY KEY UNIQUE NOT NULL, name TEXT NON NULL, symbol TEXT)'''
create_characters = '''CREATE TABLE IF NOT EXISTS characters( url TEXT PRIMARY KEY UNIQUE NOT NULL, name TEXT NON NULL, dead INT NON NULL DEFAULT 0)'''
create_allegeances = '''CREATE TABLE IF NOT EXISTS allegeances( url_character TEXT NOT NULL, url_house TEXT NOT NULL, PRIMARY KEY (url_character, url_house), FOREIGN KEY (url_character) REFERENCES characters(url), FOREIGN KEY (url_house) REFERENCES houses(url))'''

First we create the table containing the houses

conn.execute(create_homes)

Then the characters

conn.execute(create_characters)

And finally, the joining table

conn.execute(create_allegeances)

<Cursor at 0x7fe6cbd022d0>

Fill in the tables

Now, it's time to build tuples representing the data that we wish to insert into our tables. We do this through loops and comprehensions.

In [ ]:
inputs_houses = [(house["url"], house["name"], house["coatOfArms"]) for house in houses]
inputs_personnages = [(perso["url"], perso["name"], perso["died"] == "") for perso in personnages if perso["name"] != ""] # If the character has a date of death, the third element of the tuple equals Falseinputs_allegeances = [] 
# Exercise: try to transform this loop into comprehensions for perso in characters: if perso["name"] != "": for house in perso["allegiances"]: inputs_allegeances = inputs_allegeances +[(perso["url"], maison)]

After these few manipulations, we are now ready to fill our database! By reading the documentation for the sqlite3 module, we can realize that, if we have a tuple list containing
the values that we want to insert in the database, we can use the executemany() statement on the connection to insert them as one statement in the table.

In [ ]:
conn.executemany('''INSERT INTO maisons VALUES (?, ?, ?)''', inputs_houses)
conn.executemany('''INSERT INTO personnages VALUES (?, ?, ?)''', inputs_characters)
conn.executemany('''INSERT INTO allegeances VALUES (?, ?)''', inputs_allegeances)
conn.commit()

There you go. We now have the information in the database. Notice that the boolean value indicating whether the person is still has been automatically converted to 0 (for False) and 1 (for true).

Querying the database

The peculiarity of the *sqlite3* library is that; the response of an *execute()* statement is always a pointer (i.e. kind of a link to the responses returned). This means that to obtain a table with
responses to a SELECT instruction, the function method *.fetchall()* must be applied on the pointer to convert all the rows of the results in a table of tuples. Let's illustrate this
with an example: we will use the SELECT instruction to find the url (which is the primary key) of the Stark (of Winterfell) houses, Lannister (of Casterly Rock) and Targaryen (of King's Landing). This will be useful to record the tweets we have extracted.

In [ ]:
main_houses_query = '''SELECT name, url FROM houses WHERE name LIKE "%Stark of Winterfell" OR name LIKE "%Lannister of Casterly Rock" OR name LIKE "%Targaryen of King's Landing"'''

In [ ]:
res = conn.execute(main_houses_query)urls_main_houses = res.fetchall()main_houses = {main_house[0]: main_house[1] for main_house in urls_main_houses}

The next (and last step for today) is to pass the Tweets we retrieved from the Sentiment Analysis Library and to insert them in a new database table.

## Sentiment analysis

We can now pass on the tweets we've collected in a
Sentiment analysis bookstore. Sentiment analysis is a task
extremely complex and still evolving. Indeed, many
of clues to discover the meaning of a sentence (this includes by
example, the use of punctuation, certain key words, certain
emoji and sometimes complex probabilistic models - think by
example to tweets containing sarcastic remarks, the most
complex to interpret).

Fortunately, there are, once again, libraries that allow to interpret the strings passed to them and to extract from them
sentiment indicators.

One such bookstore is [Vader (Valencia Aware Dictionary and sEntiment Reasoner)](https://github.com/cjhutto/vaderSentiment). This library can be installed by the usual means (using PiPy for example, or conda).

The library is provided with some datasets so that you can
can convince you of the strength of the concepts behind it. Once convinced, it's time to apply the library to the Tweets that
we extracted.

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzeranalyser = SentimentIntensityAnalyzer() 
# We create an object that will analyze tweetstweets_texts = [(house, tweet.text) for house, tweets in tweets_on_houses.items() for tweet in tweets]tweets_texts = list(set(tweets_texts)) # This is a little trick to remove duplicate lines (there may be retweets that we are not interested in here)abrevs = {"HouseLannister": "House Lannister of Casterly Rock", "HouseStark": "House Stark of Winterfell", "HouseTargaryen": "House Targaryen of King's Landing"}tweets_with_primary_key = [(houses_importantes[abrevs[tweet[0]]], tweet[1]) for tweet in tweets_texts]load_for_database = []for tweet in tweets_with_primary_key: # Here, we use a loop because the algorithm that determines the sentiment can take a long time to run: we may prefer to run it only once per tweet sentiment = analyze.polarity_scores(tweet[1]) # We pass the text to the library, it returns a dictionary containing 3 keys: "pos", "neg", "neu" and "compound" res = (tweet[0], tweet[1], sentiment["compound"], sentiment["pos"], sentiment["neu"], sentiment["neg"]) load_for_database = load_for_database + [res]load_for_database[0:5]

*[('https://www.anapioficeandfire.com/api/houses/229', 'Exclusive #GameofThrones Silver #IronThrone Tankard\n\nOrder Now {https://t.co/xWIe02Nfnd}\n\n#affiliate… https://t.co/R0c2OvGKf4', 0.128, 0.143, 0.857, 0.0), ('https://www.anapioficeandfire.com/api/houses/229', '#HouseLannister 🦁 https://t.co/2NIYvQ5PUx', 0.0, 0.0, 1.0, 0.0), ('https://www.anapioficeandfire.com/api/houses/378', 'Esqueci a poha da #Gameofthrones #GOT #JonSnow #DaenerysTargaryen #ForTheThrone #HouseTargaryen', 0.0, 0.0, 1.0, 0.0), ('https://www.anapioficeandfire.com/api/houses/362', '@Maisie_Williams Best scene ever. Of all the series. I’ve been waiting since Arya met Gendry. It was all I ever hop… https://t.co/ISQ37QN5sb', 0.6369, 0.174, 0.826, 0.0), ('https://www.anapioficeandfire.com/api/houses/378', 'Will the real John Snow ❄️ please stand up @GameOfThrones #GamefThrones #HouseStark #HouseTargaryen https://t.co/EZZbxokFvF', 0.3182, 0.15, 0.85, 0.0)]*

Sentiment analysis returns 3 values: "proportions of beliefs" that the message is positive, negative or neutral. The sum of these three
In addition to these three values, there is a fourth, the composite index. This "summarizes" the other three into a value that is unconstrained (i.e. it can be
positive or negative). A value of the composite indicator close to 0
indicates a rather neutral tweet, a positive value indicates a tweet
generally positive and a negative value a tweet generally
negative. It is this value that will be mainly useful in the case of
analysis of the results (taking, for example, the average of all tweets on a house before and after an episode).

OK, as you can probably see (it depends on when you run this code), the algorithm is not flawless (especially since, in the
of the show, talking about death, destruction, etc. is not
necessarily negative, whereas in general, a tweet talking about these
topics is of some concern). However, for this example, as we
are not too concerned about the analysis, it will not pose too much of a problem.
problem (the graphs for the previous course will be just enough for the
sad).

Note that, if we wanted to track the interest of the twittosphere for each house per day, we could run some cells of this notebook but there are some others that should be avoided. A large part of the work consists in taking and adapting the contents of this notebook for
extract the interesting parts for your project.

Well, now, it's a matter of creating a table in our database where we'll store the tweets and their analysis. We'll just go in enter the tuples we created earlier. We'll add a column to represent the day (in fact, the exact time) we added the tweets. This will allow us to analyze the average feeling about each of the houses this week and after.
next week's episode to see if there's any developments.

In [ ]:
table_tweets_creation = '''CREATE TABLE IF NOT EXISTS tweets( house TEXT NOT NULL, content TEXT NOT NULL, composite_index NUMERIC, positive_index NUMERIC, neutral_index NUMERIC, negative_index NUMERIC, insert_when TEXT DEFAULT (date('now', 'localtime')), PRIMARY KEY (house, content), FOREIGN KEY (house) REFERENCES houses(url))'''conn.execute(creation_table_tweets)

Note the line *"insert_when TEXT DEFAULT (date('now', 'localtime')), "*.
It indicates that, if we don't specify a date for the insertion, we will just add the string corresponding to the date of the system on which the script is being run (this implies that, if we run this script at 2:00 a.m. Belgian time. simultaneously on two computers, one in Brussels and one in Buenos Aires, we will not have the same line in the databases).

That's it, we can finally insert our tweets and their information in the database. This will conclude the presentation of libraries needed to complete the second part of the work.

In [ ]:
insert_into_tweets = '''INSERT INTO tweets(house, content, composite_index, positive_index, neutral_index, negative_index) VALUES (?,?,?,?,?,?)'''conn.executemany(insert_into_tweets, load_for_database)conn.commit()

In this case, since we do not provide a tuple of the table size (the table has 7 columns but we only provide 6 values because we want to make the database add today's date), it is
It is necessary to name the columns in which the 6 values should be inserted.

The *conn.commit()* writes the changes to the database. It's worth to ensure that any changes contained in the notebook have been well written. Do not hesitate to look in the database of
given to see what the code in this book has produced.